In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import time 
import pandas as pd

os.environ["SPOTIPY_CLIENT_ID"] = ""
os.environ["SPOTIPY_CLIENT_SECRET"] = ""
os.environ["SPOTIPY_REDIRECT_URI"] = ""

SCOPES = [""]
HOME_PATH = os.path.expanduser("~")
# OAuth
class SpotifyClient:
    def __init__(self):
        self.client = spotipy.Spotify(
            auth_manager=SpotifyOAuth(
                scope=SCOPES, open_browser=False, cache_path=HOME_PATH + "/.spotify"
            ),
            client_credentials_manager=SpotifyClientCredentials(),
        )

    def get_track_ids(self, res):
        current_tracks = []
        current_tracks_ids = []

        current_tracks.extend(res["items"])

        for track in current_tracks:
            if track["track"]:
                track_uri = track["track"]["uri"]
                current_tracks_ids.append(track_uri)

        return current_tracks_ids
    
spotify_client = SpotifyClient()

In [ ]:
def get_playlists(user):
    mirror_res = spotify_client.client.user_playlists(
                user=user,
            )
    

    #current_tracks_ids = spotify_client.get_track_ids(mirror_res)
    batches_to_add = pd.DataFrame()
    batches_to_add = pd.concat((batches_to_add, pd.DataFrame(mirror_res["items"])[["id", "name", "owner", "description"]]))
    
    
    while mirror_res["next"]:
        time.sleep(1)
        mirror_res = spotify_client.client.next(mirror_res)
        batches_to_add = pd.concat((batches_to_add, pd.DataFrame(mirror_res["items"])[["id", "name", "owner", "description"]]))
        
    return batches_to_add
            


In [ ]:
#get_playlists("")

In [ ]:
mirror_res = spotify_client.client.playlist_tracks(
                    playlist_id=f"spotify:playlist:xxx",
                    limit=100,
                    fields="next, items(track(name, uri))",
                )

current_tracks_ids = spotify_client.get_track_ids(mirror_res)
batches_to_add = []
for t in current_tracks_ids:
    batches_to_add.append(t)

while mirror_res["next"]:
    time.sleep(1)
    mirror_res = spotify_client.client.next(mirror_res)
    current_tracks_ids = spotify_client.get_track_ids(mirror_res)
    for t in current_tracks_ids:
        batches_to_add.append(t)

In [ ]:
batches_to_add

In [ ]:
#Features
# FRom image
# Ngrams of title and artist
# Release date
# Duration_ms

#batches_to_add[0] = "spotify:track:"

In [ ]:
def track_to_dataframe(track):
    track_name = track["name"]
    album_name = track["album"]["name"]
    duration = track["duration_ms"]
    explicit = track["explicit"]
    release_date = track["album"]["release_date"]
    img_url = track["album"]["images"][0]["url"]
    
    artists = track["artists"]
    artist_list = []


    for artist in artists:
        artist_list.append(artist["name"])

    artist_list.sort()
    track_df = pd.DataFrame(data={"track_name": track_name,
                             "album_name": album_name,
                             "duration": duration,
                             "artists": " ".join(artist_list),
                             "explicit": explicit,
                             "release_date": pd.to_datetime(release_date),
                             "img_url": img_url,
                             "label": False
                            },  index=[0])
    return track_df

i = 0
for uri in batches_to_add:
    i+=1
    track = spotify_client.client.track(uri)

    track_df = track_to_dataframe(track)
   
    track_search = spotify_client.client.search(f"{track_df['artists'][0]} {track_df['track_name']}")["tracks"]["items"]

    for track in track_search:
        track_df = pd.concat((track_to_dataframe(track), track_df)).reset_index(drop=True)
    
 
    track_df = track_df.drop_duplicates(keep="last")
    track_df.at[track_df.index[-1], "label"] = True
    #track_df = track_df.drop_duplicates(subset=track_df.columns.difference(['artists']))

    track_df.to_csv(f"./data/{i}.csv")
